In [1]:
!pip install flask pyngrok --quiet

In [2]:
!pip install pytz


In [3]:
# Install dependencies
!pip install flask pyngrok pytz requests --quiet

# Imports
from flask import Flask, request, Response
from datetime import datetime
from pytz import timezone
from pyngrok import ngrok
import threading
import requests
import os
from collections import defaultdict

# Globals
log_file = "/content/open_log.txt"
open_counts = defaultdict(int)

app = Flask(__name__)

@app.route('/track')
def track():
    email_id = request.args.get('id', 'unknown')
    ip = request.headers.get('X-Forwarded-For', request.remote_addr)

    # Get current time in IST
    utc_now = datetime.utcnow()
    ist = timezone('Asia/Kolkata')
    ist_time = utc_now.replace(tzinfo=timezone('UTC')).astimezone(ist)
    timestamp = ist_time.strftime('%Y-%m-%d %H:%M:%S %Z')

    # Increment open count
    open_counts[email_id] += 1

    # Log info (no location)
    with open(log_file, "a") as f:
        f.write(
            f"Email ID: {email_id}\n"
            f"Opened at: {timestamp}\n"
            f"IP Address: {ip}\n"
            f"Open Count: {open_counts[email_id]}\n\n"
        )

    # Return 1x1 transparent GIF
    pixel = (
        b'\x47\x49\x46\x38\x39\x61\x01\x00\x01\x00\x80\x00\x00\x00\x00\x00'
        b'\xff\xff\xff\x21\xf9\x04\x01\x00\x00\x00\x00\x2c\x00\x00\x00\x00'
        b'\x01\x00\x01\x00\x00\x02\x02\x4c\x01\x00\x3b'
    )
    return pixel, 200, {'Content-Type': 'image/gif'}

# ✅ Web Dashboard route (no location column)
@app.route('/dashboard')
def dashboard():
    if not os.path.exists(log_file):
        return "No logs yet."

    with open(log_file, 'r') as f:
        log_data = f.read()

    # Format as HTML table
    entries = log_data.strip().split('\n\n')
    table_rows = ""
    for entry in entries:
        lines = entry.split('\n')
        values = [line.split(': ', 1)[1] if ': ' in line else '' for line in lines]
        if len(values) >= 4:
            table_rows += f"<tr><td>{values[0]}</td><td>{values[1]}</td><td>{values[2]}</td><td>{values[3]}</td></tr>"

    html = f"""
    <html>
    <head>
        <title>Tracking Dashboard</title>
        <style>
            table {{border-collapse: collapse; width: 100%;}}
            th, td {{border: 1px solid black; padding: 8px; text-align: left;}}
            th {{background-color: #f2f2f2;}}
        </style>
    </head>
    <body>
        <h2>Email Tracking Dashboard</h2>
        <table>
            <tr><th>Email ID</th><th>Opened At</th><th>IP Address</th><th>Open Count</th></tr>
            {table_rows}
        </table>
        <br>
        <a href="/export">📥 Download CSV</a>
    </body>
    </html>
    """
    return html

# ✅ Export logs as CSV (no location column)
@app.route('/export')
def export_logs():
    if not os.path.exists(log_file):
        return "No logs yet."

    with open(log_file, 'r') as f:
        log_data = f.read()

    # Convert to CSV format
    entries = log_data.strip().split('\n\n')
    csv_data = "Email ID,Opened At,IP Address,Open Count\n"
    for entry in entries:
        lines = entry.split('\n')
        values = [line.split(': ', 1)[1] if ': ' in line else '' for line in lines]
        if len(values) >= 4:
            csv_data += f"{values[0]},{values[1]},{values[2]},{values[3]}\n"

    return Response(
        csv_data,
        mimetype="text/csv",
        headers={"Content-Disposition": "attachment;filename=open_log.csv"}
    )

# Start Flask app in thread
def run_flask():
    app.run(port=5001)

threading.Thread(target=run_flask).start()

# Setup ngrok
ngrok.set_auth_token("2zLuCtN3zALATwZlVc2yps42Tj2_3nPDhDQ5GpJtJPZc7A5Tx")
public_url = ngrok.connect(5001)
print("✅ Tracking Pixel URL:")
print(public_url.public_url + "/track?id=user123")
print("📊 Dashboard URL:")
print(public_url.public_url + "/dashboard")

# Send email



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit


✅ Tracking Pixel URL:                                                                               
https://df2537f4f40a.ngrok-free.app/track?id=user123
📊 Dashboard URL:
https://df2537f4f40a.ngrok-free.app/dashboard


t=2025-11-13T01:09:15+0530 lvl=warn msg="failed to check for update" obj=updater err="Post \"https://update.equinox.io/check\": context deadline exceeded"
127.0.0.1 - - [13/Nov/2025 01:09:18] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [13/Nov/2025 01:09:19] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

sender_email = "praneethtirumalasetty811@gmail.com"
receiver_email = "mohammednorulashik@gmail.com"
app_password = "**** **** **** ****"

msg = MIMEMultipart("alternative")
msg["Subject"] = "Tracked Email"
msg["From"] = sender_email
msg["To"] = receiver_email

html = f"""
<html>
  <body>
    <p>Hello!</p>
    <p>This email contains a tracking pixel.</p>
    <img src="https://3ff262974445.ngrok-free.app/track?id=mohammednorulashik@gmail.com" width="1" height="1" style="display:none;" />
    <p>If image is blocked, click <a href="https://3ff262974445.ngrok-free.app/track?id=mohammednorulashik@gmail.com">this link</a>.</p>
  </body>
</html>
"""

msg.attach(MIMEText(html, "html"))

with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
    server.login(sender_email, app_password)
    server.sendmail(sender_email, receiver_email, msg.as_string())

print("📤 Email sent.")

SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials d2e1a72fcca58-7b73ae09668sm4486104b3a.69 - gsmtp')

t=2025-11-13T02:25:17+0530 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=ac46ceea4a11 err="read tcp [2402:3a80:1bbf:ed:976:3648:a8da:2fda]:54964->[2600:1f16:d83:1202:dec4:7096:ae88:6144]:443: wsarecv: An established connection was aborted by the software in your host machine."
t=2025-11-13T02:25:17+0530 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=4eb792f17d2c clientid=b303b327e1037ff15a59894422fadc4c
t=2025-11-13T02:25:27+0530 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="failed to dial ngrok server with address \"connect.us.ngrok-agent.com:443\": dial tcp 3.148.88.97:443: i/o timeout"
t=2025-11-13T02:25:37+0530 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="failed to dial ngrok server with address \"connect.us.ngrok-agent.com:443\": dial tcp 3.148.88.97:443: i/o timeout"
t=2025-11-13T02:25:48+0530 lvl=eror msg="failed to reconnect session" obj=tunnels.session

In [5]:
!cat /content/open_log.txt


Email ID: nukesshere@gmail.com
Opened at: 2025-07-10 15:26:24 IST
IP Address: 2406:7400:c8:c2ec:1529:4da2:5395:c5a3
Open Count: 2

Email ID: user123=nukesshere@gmail.com
Opened at: 2025-07-10 15:33:06 IST
IP Address: 2406:7400:c8:c2ec:1529:4da2:5395:c5a3
Open Count: 1

Email ID: mohammednorulashik@gmail.com
Opened at: 2025-07-10 15:34:07 IST
IP Address: 2406:7400:c8:c2ec:1529:4da2:5395:c5a3
Open Count: 1



In [4]:
# Clear previous logs
open("/content/open_log.txt", "w").close()
print("✅ Log file cleared.")


✅ Log file cleared.
